In [1277]:
!pip install sympy numpy matplotlib scipy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1278]:
import numpy as np
import sympy as sp
import matplotlib as plt
import scipy as sc

This script aids the creation of a Sedra high pass notch filter
Setup:

In [1279]:
# Desired parameters:
Q = 5.25
fp = 3667
fz = 13794

# Op-Amp parameters
GBWP = 1000000

# since this is a high pass notch filter, n1 has to be 0 so that transmission zeros are possible
n1 = 0

# since the gain of the system when the frequency goes to infinity has to be 1 for it to be normalized, n2 has to be 1 over the overshoot Q
n2 = 0.028

1) The value of Qo has to be chosen according to the design parameters since it depends on the sensibility of the components.

In [1280]:
Qo = 2

2) The parameters K, k and n are then calculated from the previous parameters

In [1281]:
K = 1 + ((1/(2*Qo**2))*(1 - (Qo/Q)))
k = n2 / (1 - (Qo/Q))
n = k*(1 - (Qo/(K*Q)))

display("K: {}".format(K))
display("k: {}".format(k))
display("n: {}".format(n))


'K: 1.0773809523809523'

'k: 0.04523076923076923'

'n: 0.029237569060773478'

The user must then pick a value for C41, and then select a value for C42 that fulfills the condition

In [1282]:
C41 = 67e-12

C42 = C41 * (n/(1 - n))
display("Maximum C42 value: {}".format(C42))

'Maximum C42 value: 2.0179160880551825e-12'

In [1283]:
C42 = 2e-12
display("C42 value: {}".format(C42))

C1 = 4*(Qo**2)*(C41 + C42)
display("Max C1 value: {}".format(C1))

'C42 value: 2e-12'

'Max C1 value: 1.1040000000000001e-09'

The user must then introduce a nominal value lower than the max C1

In [1284]:
C1 = 1e-9

In [1285]:
wp = 2*np.pi*fp*(1 + Qo * (fp / GBWP))
Qp = Q*(1 - 2*Qo*Q*(fp/GBWP)*((1/(2*Q)) - (fp/GBWP)))

display("Wp value: {}".format(wp))
display("Qp value: {}".format(Qp))

'Wp value: 23209.41911221169'

'Qp value: 5.21297901951225'

In [1286]:
G3 = G2 = wp*(np.sqrt(C1*(C41 + C42)))

display("G3 = G2 = {}".format(G2))

'G3 = G2 = 6.096615647333733e-06'

Then the user must select a convenient value for Gb

In [1287]:
Rb = 100
Gb = 1/Rb

Ga = Gb*(((C41 + C42)/C1)*((G2 + G3)/G3)-(wp*(C41 + C42)/(Qp*G3)))
display("Ga value: {}".format(Ga))

'Ga value: 0.0008761067179667044'

In [1288]:
Ga1s, Ga2s, Gas, Gbs, C42s, C41s, G21s, G22s, G3s, C1s, G2s = sp.symbols('Ga1, Ga2, Ga, Gb, C42, C41, G21, G22, G3, C1, G2')
lhs = ((Ga1s + Ga2s + Gbs)/Gbs)*(C42s/(C41s + C42s))*((G2s + G3s)/C1s)
rhs = (Ga2s/Gbs)*((G3s/(C41s + C42s)) + ((G2s + G3s)/C1s))

eq1 = sp.Eq(lhs, rhs)

eq2 = sp.Eq(Ga1s + Ga2s, Gas)

display(eq1)
display(eq2)

eq1 = eq1.subs(C42s, C42).subs(C41s, C41).subs(Gbs, Gb).subs(C1s, C1).subs(G3s, G3).subs(G2s, G2)
eq2 = eq2.subs(Gas, Ga)

solutions = sp.solve((eq1, eq2), (Ga1s, Ga2s))

Ga1 = solutions[Ga1s]
Ga2 = solutions[Ga2s]

display("Ga1:")
display(Ga1)

display("Ga2:")
display(Ga2)

Eq(C42*(G2 + G3)*(Ga1 + Ga2 + Gb)/(C1*Gb*(C41 + C42)), Ga2*(G3/(C41 + C42) + (G2 + G3)/C1)/Gb)

Eq(Ga1 + Ga2, Ga)

'Ga1:'

0.000837877871857858

'Ga2:'

3.82288461088458e-5

In [1289]:
new_n2 = ((Ga1 + Ga2 + Gb)/Gb)*(C42/(C41 + C42)) - (Ga2/Gb)
display("New n2")
display(new_n2)

'New n2'

0.0277020623977145

In [1290]:
n2s, wzs = sp.symbols('n2, wz')

lhs = n2s * wzs**2
rhs = (G3s * G2s) / (C1s * (C41s + C42s)) * ((G22s / (G21s + G22s)) * ((Ga1s + Ga2s + Gbs) / Gbs) - (Ga2s / Gbs))

eq1 = sp.Eq(lhs, rhs)

display(eq1)

eq1 = eq1.subs(G3s, G3).subs(Ga1s, Ga1).subs(Ga2s, Ga2).subs(Gbs, Gb).subs(C1s, C1).subs(C41s, C41).subs(C42s, C42).subs(n2s, new_n2).subs(wzs, 2*np.pi*fz).subs(G2s, G2)

eq2 = sp.Eq(G21s + G22s, G2s)

display(eq2)

eq2 = eq2.subs(G2s, G2)

solutions = sp.solve((eq1, eq2), (G21s, G22s))

display(solutions[G21s])
display(solutions[G22s])

display("R21: {}".format(1/solutions[G21s]))
display("R22: {}".format(1/solutions[G22s]))
display("Ra1: {}".format(1/Ga1))
display("Ra2: {}".format(1/Ga2))
display("R3: {}".format(1/G3))
display("Rb: {}".format(1/Gb))
display("C41: {}".format(C41))
display("C42: {}".format(C42))
display("C1: {}".format(C1))
display("N2: {}" .format(new_n2))

Eq(n2*wz**2, G2*G3*(G22*(Ga1 + Ga2 + Gb)/(Gb*(G21 + G22)) - Ga2/Gb)/(C1*(C41 + C42)))

Eq(G21 + G22, G2)

3.90978160787263e-6

2.18683403946110e-6

'R21: 255768.761607151'

...

'Rb: 100.0'

'C41: 6.7e-11'

'C42: 2e-12'

'C1: 1e-09'

'N2: 0.0277020623977145'